<a href="https://colab.research.google.com/github/vadize/vadize.github.io/blob/main/FinalModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00
Cloning into 'korean-malicious-comments-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 30 (delta 8), reused 2 (delta 2), pack-reused 21
Receiving objects: 100% (30/30), 525.19 KiB | 1.50 MiB/s, done.
Resolving deltas: 100% (15/15), done.
device: cuda:0


In [ ]:
df = pd.read_csv("/content/korean-malicious-comments-dataset/Dataset.csv", sep="\t")


In [ ]:
null_idx = df[df.lable.isnull()].index


In [ ]:

df.loc[null_idx, "lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])
#d
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[:-2])

In [ ]:
df = df.astype({"lable":"int"})

In [ ]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [ ]:
# remove redundant data
print('before deleting redundant data : {}'.format(len(train_data)))
train_data.drop_duplicates(subset=["content"], inplace= True)
print('after deleting redundant data  : {}'.format(len(train_data)))


print('before deleting redundant data  : {}'.format(len(test_data)))
test_data.drop_duplicates(subset=["content"], inplace= True)
print('after deleting redundant data  : {}'.format(len(test_data)))

before deleting redundant data : 8000
after deleting redundant data  : 7992
before deleting redundant data  : 2000
after deleting redundant data  : 2000


In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
#tokenized

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

In [ ]:
#dataset

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_label = train_data["lable"].values
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_label = test_data["lable"].values
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
!pip uninstall transformers
!pip uninstall accelerate
!pip install transformers[torch]
!pip install accelerate


Found existing installation: transformers 4.34.1
Uninstalling transformers-4.34.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.34.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.34.1
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00


In [ ]:
training_args = TrainingArguments(
    output_dir='./',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,                         # Transformers model
    args=training_args,                  # Defined Training Arguments
    train_dataset=train_dataset,         # Train dataset
    eval_dataset=test_dataset,           # Test dataset
    compute_metrics=compute_metrics,     # metrics
)

In [ ]:
trainer.train()

<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.411700
1000,0.330500
1500,0.220000
2000,0.248000
2500,0.126700
3000,0.134900
3500,0.065500
4000,0.078700
4500,0.052700
5000,0.038400


<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=9990, training_loss=0.09481407080565368, metrics={'train_runtime': 2173.0201, 'train_samples_per_second': 36.778, 'train_steps_per_second': 4.597, 'total_flos': 5256958886092800.0, 'train_loss': 0.09481407080565368, 'epoch': 10.0})

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)


<ipython-input-14-b5e02a6514c5>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.7313545346260071,
 'eval_accuracy': 0.9055,
 'eval_f1': 0.9046898638426625,
 'eval_precision': 0.8996990972918756,
 'eval_recall': 0.90973630831643,
 'eval_runtime': 14.1216,
 'eval_samples_per_second': 141.627,
 'eval_steps_per_second': 2.266,
 'epoch': 10.0}

In [ ]:
output_dir = '/content/drive/My Drive'
trainer.save_model(output_dir)


In [ ]:
import os

# List the contents of the output directory in Google Drive
os.listdir(output_dir)


['pytorch_model.bin', 'config.json', 'training_args.bin']

In [ ]:
from transformers import AutoModelForSequenceClassification

# Define the path to the saved model in your Google Drive
model_path = '/content/drive/My Drive'

# Load the saved model from Google Drive
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")  #
input_text = "^ㅣ발"
tokenized_input = tokenizer(input_text, return_tensors="pt")


In [ ]:

with torch.no_grad():
    outputs = loaded_model(**tokenized_input)
    predicted_class = torch.argmax(outputs.logits, dim=1)

# 'predicted_class' now contains the predicted class label


In [ ]:
# Define a mapping from class indices to class labels
class_mapping = {0: "욕설", 1: "정상"}

# Get the predicted class index
predicted_class_index = predicted_class.item()  # Convert to a Python integer

# Map the predicted index to the corresponding class label
predicted_class_label = class_mapping[predicted_class_index]

# Print the predicted class label
print("Predicted Class Label:", predicted_class_label)


Predicted Class Label: 욕설
